We see that there are many missing values and they can be taken from the internet. I can proceed in that direction.

In [57]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import mysql.connector

In [58]:
df_netflix_movie_data = pd.read_csv('./Datasets/netflix_titles.csv', lineterminator = '\n')
df_prime_movie_data = pd.read_csv('./Datasets/amazon_prime_titles.csv', lineterminator = '\n')
df_disney_plus_movie_data = pd.read_csv('./Datasets/disney_plus_titles.csv', lineterminator = '\n')
df_hulu_movie_data = pd.read_csv('./Datasets/hulu_titles.csv', lineterminator = '\n')

In [71]:
missing_value_netflix = df_netflix_movie_data.isna().sum()
print(missing_value_netflix)

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        98
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [72]:
print("Number of records in Netflix DataFrame:", df_netflix_movie_data.shape[0])

Number of records in Netflix DataFrame: 8807


In [73]:
missing_value_prime = df_prime_movie_data.isna().sum()
print(missing_value_prime)

show_id            0
type               0
title              0
director        2083
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64


In [ ]:
print("Number of records in Prime Video DataFrame:", df_prime_movie_data.shape[0])

In [74]:
missing_value_disney_plus = df_disney_plus_movie_data.isna().sum()
print(missing_value_disney_plus)

show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
dtype: int64


In [ ]:
print("Number of records in Disney Plus DataFrame:", df_disney_plus_movie_data.shape[0])

In [75]:
missing_value_hulu = df_hulu_movie_data.isna().sum()
print(missing_value_hulu)

show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
dtype: int64


In [ ]:
print("Number of records in Hulu DataFrame:", df_hulu_movie_data.shape[0])

As we can see clearly, above only columns director, cast, country, date_added, rating, duration. Rather than deleting them. If we can get those values back in the dataset it would be best.
I can use information present on the website imdb to get the data and fill it. I can use web scrapping to get the data and fill it in the dataset.

But before that I can check data. One dataset row may have data of the for the other dataset missing row values. Let's first check for the 

In [59]:
print("Netflix DataFrame Column Data Types:")
print(df_netflix_movie_data.dtypes)

Netflix DataFrame Column Data Types:
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [69]:
print("\nPrime Video DataFrame Column Data Types:")
print(df_prime_movie_data.dtypes)


Prime Video DataFrame Column Data Types:
show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      datetime64[ns]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object


In [68]:

print("\nDisney Plus DataFrame Column Data Types:")
print(df_disney_plus_movie_data.dtypes)


Disney Plus DataFrame Column Data Types:
show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      datetime64[ns]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object


In [67]:
print("\nHulu DataFrame Column Data Types:")
print(df_hulu_movie_data.dtypes)


Hulu DataFrame Column Data Types:
show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      datetime64[ns]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object


Okay I will be first converting the datatypes of the columns of the dataframes according to the requirement and then after that I will be tranferring data to the SQL. Through this I can ensure that there is no error while tranferring the data.

Why I want to transfer the data to SQL?
I currently want to fill the missing values of the columns director, cast, country, date_added, rating and duration in all datasets. But there are chances that missing value of 1 dataset might be present in another dataset. So for doing that in optimized manner I am using the SQL. 
Other approaches:

- I can loop through datasets to find values. But this approach will take most time as there are 4 datasets and time taken will get to O($n^4$). This approach is not great.
- I can use Hash tables for this but it will be take most of the memory for it. It would be better to go for Database as I would be taking it's help for data wrangling also.

In [63]:
dataframes = [df_netflix_movie_data, df_prime_movie_data, df_disney_plus_movie_data, df_hulu_movie_data]

In [64]:
for df in dataframes:
    df["show_id"] = df["show_id"].astype("string")  # Explicit string type
    df["type"] = df["type"].astype("string")
    df["title"] = df["title"].astype("string")
    df["director"] = df["director"].astype("string")
    df["cast"] = df["cast"].astype("string")  # Convert float64 to string
    df["country"] = df["country"].astype("string")
    df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")  # Convert to Date
    df["release_year"] = df["release_year"].astype("Int64")  # Explicitly use Int64
    df["rating"] = df["rating"].astype("string")
    df["duration"] = df["duration"].astype("string")
    df["listed_in"] = df["listed_in"].astype("string")
    df["description"] = df["description"].astype("string")



In [ ]:
# Verify Data Types
for i, df in enumerate(dataframes, start=1):
    print(f"\nData Types for DataFrame {i}:\n", df.dtypes)

In [66]:
print("Netflix DataFrame Column Data Types:")
print(df_netflix_movie_data.dtypes)

Netflix DataFrame Column Data Types:
show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      datetime64[ns]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object


In [89]:
df_netflix_movie_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,<NA>,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,<NA>,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",<NA>,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,<NA>,<NA>,<NA>,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,<NA>,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [90]:
df_hulu_movie_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,<NA>,<NA>,<NA>,2021-10-24,2021,TV-MA,<NA>,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,<NA>,<NA>,<NA>,2021-10-23,2020,<NA>,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,<NA>,<NA>,<NA>,2021-10-23,2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,<NA>,<NA>,<NA>,2021-10-22,2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,<NA>,<NA>,<NA>,2021-10-22,2021,<NA>,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [79]:
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="Himanshu",
    password="Hustling@2000",
)
cursor = conn.cursor()

In [80]:
cursor.execute("CREATE DATABASE IF NOT EXISTS movies_db;")
cursor.close()
conn.close()

In [83]:
conn = mysql.connector.connect(
    host="localhost",
    user="Himanshu",
    password="Hustling@2000",
    database="movies_db"
)
cursor = conn.cursor()

In [88]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS movies_data (
        show_id VARCHAR(255),
        type VARCHAR(50),
        title VARCHAR(500),
        director VARCHAR(300),
        cast VARCHAR(2000),
        country VARCHAR(100),
        date_added DATE,
        release_year YEAR,
        rating VARCHAR(50),
        duration VARCHAR(100),
        listed_in VARCHAR(300),
        description TEXT,
        PRIMARY KEY (title, release_year)
    )
""")